In [15]:
import nrrd
import napari
import os
import numpy as np
import zarr
import blosc2
from helper import *

In [16]:
# Data location and size parameters
scroll_name = 's1'
x = 3500
y = 2500
z = 1000
chunk_size = 256
pad_amount = 100
current_directory = os.getcwd()
pad_state = False

In [17]:
#---Zarr specific code, comment out if not using and set using_zarr to false---
using_zarr = True #change to false if using nrrd or other format
s1_zarr_path = "/Volumes/16TB_RAID_0/Scroll1/Scroll1.zarr" #change this to the path of the zarr file if using zarr
s1_zarr_multi_res = zarr.open(s1_zarr_path, mode='r')
s1_zarr = s1_zarr_multi_res[0]

zarr_data = s1_zarr[z:z+chunk_size, y:y+chunk_size, x:x+chunk_size]

#Note: will crash if out of bounds and not checking at the moment
padded_zarr_data = s1_zarr[z-pad_amount:z+chunk_size+pad_amount, y-pad_amount:y+chunk_size+pad_amount, x-pad_amount:x+chunk_size+pad_amount]
data = zarr_data

In [18]:
# temp Code to save zarr chunk as nrrd
save_dir = current_directory+f'/data/{x}_{y}_{z}_{scroll_name}.nrrd'
print(save_dir)
# nrrd.write(save_dir, zarr_data)

#temp saved nrrd locally and load it in
data, _ = nrrd.read(save_dir)

/Users/jamesdarby/Documents/VesuviusScroll/GP/Volumetric_Vesuvius_Labelling/data/3500_2500_1000_s1.nrrd


In [19]:
# Jordi's gross volumetric labels
file_path = os.path.join(current_directory, 'data/s1_gross_labels.b2nd')
blosc2_full_array = blosc2.open(file_path, mode='r')

label_data = blosc2_full_array[z:z+chunk_size, y:y+chunk_size, x:x+chunk_size]
label_data = label_data * np.logical_not(bright_spot_mask(data))

### Custom Napari Keybinds:<br>
q to decrease brush size<br>
e to increase brush size<br>
w to select label layer that was last clicked in move mode, alternatively use color picker (4)<br>
s to toggle show selected label<br>
a to decrease selected label value<br>
d to increase selected label value<br>
arrow keys scrub through layers<br>
r or / to toggle label visibility<br>
t to toggle borders visibility<br>
f or down arrow for 20 iteration flood fill<br>
g or up arrow for 100 iteration flood fill<br>

In [20]:
#stable napari extensions

# Initialize the Napari viewer
viewer = napari.Viewer()

#layer name variables
label_name = 'Labels'
data_name = 'Data'
border_name = 'Borders'
sheet_seperations_name = 'Sheet Seperations'
ff_name = 'flood_fill_layer'
label_3d_name = '3D Label Edit Layer'
border_class = 254

# Add the 3D data to the viewer
image_layer =  viewer.add_image(data, colormap='gray', name=data_name)
labels_layer = viewer.add_labels(label_data, name=label_name)

#load saved labels and sheet seperations if they exist
file_path = 'output/volumetric_labels/'
label_path = os.path.join(current_directory, file_path, f"{x}_{y}_{z}_xyz_{chunk_size}_chunk_{scroll_name}_vol_label.nrrd")
if os.path.exists(label_path):
    label_data, _ = nrrd.read(label_path)
    label_data = label_data * np.logical_not(bright_spot_mask(data))
    labels_layer.data = label_data

padded_labels = np.pad(label_data, pad_width=pad_amount, mode='constant', constant_values=0)

sheet_seperations_path = os.path.join(current_directory, file_path, f"{x}_{y}_{z}_xyz_{chunk_size}_chunk_{scroll_name}_vol_sheet_seperations.nrrd")
if os.path.exists(sheet_seperations_path):
    data, _ = nrrd.read(sheet_seperations_path)
    viewer.add_labels(data, name=sheet_seperations_name)

#Keybind to save the labels and sheet seperations layer
@viewer.bind_key('h')
def save_labels(viewer):
    msg = 'save labels'
    viewer.status = msg
    print(msg)
    current_directory = os.getcwd()
    file_path = f'output/volumetric_labels/'
    output_path = os.path.join(current_directory, file_path)
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    print(labels_layer.data.shape, labels_layer.data.dtype)
    nrrd.write(os.path.join(output_path,f"{x}_{y}_{z}_xyz_{chunk_size}_chunk_{scroll_name}_vol_label.nrrd"), labels_layer.data)

    if sheet_seperations_name in viewer.layers:
        nrrd.write(os.path.join(output_path,f"{x}_{y}_{z}_xyz_{chunk_size}_chunk_{scroll_name}_vol_sheet_seperations.nrrd"), viewer.layers[sheet_seperations_name].data)

#keybind to toggle settings to draw the sheet seperations/border class brush
@viewer.bind_key('v')
def draw_border_class(viewer):
    msg = 'draw border class'
    viewer.status = msg
    print(msg)
    labels_layer.selected_label = border_class
    labels_layer.brush_size = 2
    labels_layer.mode = 'paint'

#run connected components on the labels layer to get instance segmentations
@viewer.bind_key('c')
def connected_components(viewer):
    msg = 'connected components'
    viewer.status = msg
    print(msg)

    #mask for the border class from the labels layer
    mask = (labels_layer.data == border_class)
    old_borders = np.zeros_like(labels_layer.data)
    old_borders[labels_layer.data == border_class] = border_class

    #new borders from the sheet seperations layer and labels layer
    new_borders = viewer.layers[sheet_seperations_name].data | old_borders
    new_borders[new_borders > 0] = border_class

    if sheet_seperations_name in viewer.layers:
        mask_2 = (viewer.layers[sheet_seperations_name].data == border_class)
        mask = mask | mask_2
        viewer.layers[sheet_seperations_name].data = new_borders
    else:
        viewer.add_labels(new_borders, name=sheet_seperations_name)

    #connected components data with both layer's borders removed
    cc_data = labels_layer.data.copy()
    cc_data[mask] = 0

    labels_layer.data = label_foreground_structures_napari(cc_data, border_class= border_class, min_size=1000)
    msg = 'connected components finished'
    viewer.status = msg
    print(msg)

#keybind to toggle the labels layer visibility
@viewer.bind_key('r')
def toggle_labels_visibility(viewer):
    msg = 'toggle labels visibility'
    viewer.status = msg
    print(msg)
    if viewer.dims.ndisplay == 3 and label_3d_name in viewer.layers:
        viewer.layers[label_3d_name].visible = not viewer.layers[label_3d_name].visible
    else:
        labels_layer.visible = not labels_layer.visible

#keybind alt to toggle the labels layer visibility
viewer.bind_key('/', toggle_labels_visibility)

#keybind to toggle the data visibility
@viewer.bind_key('.')
def toggle_data_visibility(viewer):
    msg = 'toggle data visibility'
    viewer.status = msg
    print(msg)
    image_layer.visible = not image_layer.visible

#keybind alt to toggle the data layer visibility
viewer.bind_key('t', toggle_data_visibility)

#keybind to decrease teh brush size of the labels layer
@viewer.bind_key('q')
def decrease_brush_size(viewer):
    msg = 'decrease brush size'
    viewer.status = msg
    print(msg)
    labels_layer.brush_size = labels_layer.brush_size - 1

#keybind to increase the brush size of the labels layer
@viewer.bind_key('e')
def increase_brush_size(viewer):
    msg = 'increase brush size'
    viewer.status = msg
    print(msg)
    labels_layer.brush_size = labels_layer.brush_size + 1

#keybind to toggle the show selected label only mode
@viewer.bind_key('s')
def toggle_show_selected_label(viewer):
    msg = 'toggle show selected label'
    viewer.status = msg
    print(msg)
    labels_layer.show_selected_label = not labels_layer.show_selected_label

#keybind to cycle through the selected label
@viewer.bind_key('a')
def decrease_selected_label(viewer):
    msg = 'decrease selected label'
    viewer.status = msg
    print(msg)
    labels_layer.selected_label = labels_layer.selected_label - 1

#keybind to cycle through the selected label
@viewer.bind_key('d')
def increase_selected_label(viewer):
    msg = 'increase selected label'
    viewer.status = msg
    print(msg)
    labels_layer.selected_label = labels_layer.selected_label + 1

# Function to capture cursor information when 'w' is pressed
def capture_cursor_info(event):
    # Get cursor position in world coordinates
    position = viewer.cursor.position

    # Convert world coordinates to data indices
    indices = tuple(int(np.round(coord)) for coord in position)

    # Get the value of the label under the cursor
    label_value = labels_layer.data[indices]

    # Print the cursor position and label value
    print(f"Cursor Position: {indices}, Label Value: {label_value}")
    labels_layer.selected_label = label_value

# keybind to capture cursor info and select the label under the cursor
# 4 and the color picker also works for this
@viewer.bind_key('w')
def on_w_key(event):
    capture_cursor_info(event)

#keybind to run the new label interpolation function
@viewer.bind_key('x')
def interpolate_borders(viewer):
    msg = 'interpolating borders'
    viewer.status = msg
    print(msg)
    interpolated_borders = interpolate_slices(labels_layer.data, border_class)
    if sheet_seperations_name in viewer.layers:
        viewer.layers[sheet_seperations_name].data = interpolated_borders
    else:
        viewer.add_labels(interpolated_borders, name=sheet_seperations_name)

# Add an empty labels layer for the flood fill result
flood_fill_layer = viewer.add_labels(np.zeros_like(data), name=ff_name)

# Global variable to hold the current flood fill distance
current_distance = 20

#keybind to run the flood fill function with a distance of 20
@viewer.bind_key('f')
def flood_fill(viewer, distance=20):
    msg = 'flood fill'
    viewer.status = msg
    print(msg)
    # Get the cursor position in data coordinates
    cursor_position = viewer.cursor.position
    cursor_position = tuple(int(np.round(coord)) for coord in cursor_position)

    # Get the current labels layer
    labels_layer = viewer.layers[label_name]

    # Get the current labels
    labels = labels_layer.data

    # Perform the flood fill operation
    flood_fill_result = limited_bfs_flood_fill(labels, cursor_position, distance)

    # Update the flood fill layer with the result
    flood_fill_layer.data = flood_fill_result

#keybind to run the flood fill function with a distance of 100
@viewer.bind_key('g')
def on_g_event(viewer):
    flood_fill(viewer, 100)

#keybind alt to run the flood fill function with a distance of 100
@viewer.bind_key('Up')
def on_up_arrow_event(viewer):
    flood_fill(viewer, 100)

#keybind alt to run the flood fill function with a distance of 20
@viewer.bind_key('Down')
def on_down_arrow_event(viewer):
    flood_fill(viewer, 20)

In [21]:
#Development Napari helper functions (and ones that are annoying to move to helper.py)

# Variable to store the oblique plane information
oblique_plane_info = {
    'position': (32, 32, 32),
    'normal': (1, 0, 0)
}

# Persistent variables to store the previous state and mask
previous_label_3d_data = None
manual_changes_mask = None

# Function to shift the plane along its normal vector
def shift_plane(layer, direction):
    if viewer.dims.ndisplay == 3 and layer.depiction == 'plane':
        # Get the current position and normal of the plane
        current_position = np.array(layer.plane.position)
        normal_vector = np.array(layer.plane.normal)

        # Calculate the new position
        new_position = current_position + direction * normal_vector

        # Update the plane position
        layer.plane.position = tuple(new_position)
        print(f"Shifted plane to {direction} along the normal: new position = {new_position}")
    else:
        # If in 2D mode, shift the slice by 1
        current_step = viewer.dims.current_step[0]
        new_step = current_step + (direction)
        viewer.dims.set_current_step(0, new_step)
        print(f"Shifted 2D slice to: {new_step}")

#keybind to setup the 3d viewing mode more conviniently
@viewer.bind_key('\\')
def switch_to_plane(viewer):
   # Switch to 3D mode
    if viewer.dims.ndisplay == 3:
        viewer.dims.ndisplay = 2
        for layer in viewer.layers:
            if layer.name != label_3d_name:
                viewer.layers[layer.name].visible = True
            else:
                viewer.layers[layer.name].visible = False
        viewer.layers.selection.active = viewer.layers[label_name]
        viewer.layers[label_name].contour = 1

    else:
        step_val = viewer.dims.current_step
        print(f"Current step: {step_val}")
        viewer.dims.ndisplay = 3
    
        # Hide all layers except the one named `data_name`
        for layer in viewer.layers:
            if layer.name != data_name and layer.name != ff_name:
                viewer.layers[layer.name].visible = False
            
            elif layer.name == data_name:
                # Change the depiction of `data_name` layer from volume to plane
                viewer.layers[layer.name].visible = True
                viewer.layers[layer.name].depiction = 'plane'
                viewer.layers[layer.name].plane.position = (step_val[0], 0, 0)
                viewer.layers[layer.name].affine = np.eye(3)  # Ensure the affine transform is identity for proper rendering
                viewer.layers[layer.name].blending = 'opaque'
                viewer.layers.selection.active = viewer.layers[layer.name]

def cut_label_at_plane(viewer):
    global previous_label_3d_data, manual_changes_mask

    data_plane = viewer.layers[data_name]
    if data_plane.depiction != 'plane':
        print("Please switch to plane mode by pressing '\\' key.")
        return
    
    position = data_plane.plane.position
    normal = data_plane.plane.normal
    viewer.layers[data_name].blending = 'opaque'

    # Create a meshgrid for the label data coordinates
    z, y, x = np.meshgrid(np.arange(viewer.layers[label_name].data.shape[0]),
                          np.arange(viewer.layers[label_name].data.shape[1]),
                          np.arange(viewer.layers[label_name].data.shape[2]),
                          indexing='ij')

    # Calculate the distance of each voxel from the plane
    distances = (x - position[2]) * normal[2] + (y - position[1]) * normal[1] + (z - position[0]) * normal[0]

    # Create a copy of the label data and set all voxels between the viewer and the plane to 0
    new_label_data = labels_layer.data.copy()
    new_label_data[distances > 1] = 0

    # Check if the label_3d_name layer already exists
    if label_3d_name in viewer.layers:
        existing_layer = viewer.layers[label_3d_name]
        if isinstance(existing_layer, napari.layers.Labels):
            # Calculate the manual changes mask
            if previous_label_3d_data is not None:
                manual_changes_mask = existing_layer.data != previous_label_3d_data
            
            # Apply the manual changes to the label_name layer
            label_name_layer = viewer.layers[label_name]
            label_name_layer.data[manual_changes_mask] = existing_layer.data[manual_changes_mask]

    # Remove the old label_3d_name layer if it exists
    visible_state = True
    if label_3d_name in viewer.layers:
        visible_state = viewer.layers[label_3d_name].visible
        viewer.layers.remove(viewer.layers[label_3d_name])
    
    # Add a new label layer with the updated data
    viewer.add_labels(new_label_data, name=label_3d_name)
    viewer.layers[label_3d_name].visible = visible_state

    # Store the current state of the label_3d_name layer for future comparison
    previous_label_3d_data = new_label_data.copy()

In [22]:
#dev napari extensions
pad_state = False

#keybind to shift the plane along the normal vector in 3d viewing mode
@viewer.bind_key('Left', overwrite=True)
def on_left_arrow_event(viewer):
    shift_plane(viewer.layers[data_name], -1)
    if viewer.dims.ndisplay == 3 and label_3d_name in viewer.layers and viewer.layers[label_3d_name].visible:
        cut_label_at_plane(viewer)

#keybind to shift the plane along the normal vector in 3d viewing mode
@viewer.bind_key('Right', overwrite=True)
def on_right_arrow_event(viewer):
    shift_plane(viewer.layers[data_name], 1)
    if viewer.dims.ndisplay == 3 and label_3d_name in viewer.layers and  viewer.layers[label_3d_name].visible:
        cut_label_at_plane(viewer)


# keybind to cut the label layer at the oblique plane, also called by left and right arrow
@viewer.bind_key(']')
def cut_label_at_oblique_plane(viewer):
    if viewer.dims.ndisplay == 3:
        cut_label_at_plane(viewer)
        viewer.layers[label_3d_name].visible = True

# @viewer.bind_key('y', add_padding_contextual_data(viewer, pad_state, using_zarr))
# pad_state = add_padding_contextual_data(viewer, pad_state, using_zarr)

@viewer.bind_key('i')
def add_padding_contextual_data(viewer, using_zarr=using_zarr):
    global pad_state

    if pad_state:
        if using_zarr:
            data = zarr_data
            viewer.layers[data_name].data = data
            if label_3d_name in viewer.layers:
                viewer.layers.remove(viewer.layers[label_3d_name])
            # Remove padding from the layers
            for layer in viewer.layers:
                if layer.name is not data_name:
                    original_data = layer.data.copy()
                    slices = tuple(slice(pad_amount, -pad_amount) if dim > 2 * pad_amount else slice(None) for dim in original_data.shape)
                    layer.data = original_data[slices]
            shift_plane(viewer.layers[data_name], -pad_amount)
            # viewer.dims.current_step = viewer.dims.current_step[0] - pad_amount, viewer.dims.current_step[1], viewer.dims.current_step[2]
        pad_state = False
    else:
        if using_zarr:
            data = padded_zarr_data
            viewer.layers[data_name].data = data
            if label_3d_name in viewer.layers:
                viewer.layers.remove(viewer.layers[label_3d_name])
            # Add padding to the layers
            for layer in viewer.layers:
                if layer.name is not data_name:
                    print(layer.name)
                    original_data = layer.data.copy()
                    pad_width = ((pad_amount, pad_amount), (pad_amount, pad_amount), (pad_amount, pad_amount))
                    layer.data = np.pad(original_data, pad_width=pad_width, mode='constant', constant_values=0)
            shift_plane(viewer.layers[data_name], pad_amount)
            # viewer.dims.current_step = viewer.dims.current_step[0] + pad_amount, viewer.dims.current_step[1], viewer.dims.current_step[2]
        pad_state = True
    # return pad_state

# Default napari settings for Vesuvius Volumetric Labeling
viewer.axes.visible = True
labels_layer.n_edit_dimensions = 3
labels_layer.brush_size = 5
labels_layer.opacity = 0.5
labels_layer.contour = 1

viewer.layers.selection.active = viewer.layers[label_name]

# Start the Napari event loop
napari.run()

Labels
Sheet Seperations
flood_fill_layer
Shifted 2D slice to: 27
Shifted 2D slice to: 127
Current step: (127, 127, 127)


/Users/jamesdarby/anaconda3/envs/napari_vesuvius/lib/python3.12/site-packages/napari/layers/labels/labels.py:945: UserWarning: Contours are not displayed during 3D rendering
  warnings.warn(


Labels
Sheet Seperations
flood_fill_layer
Shifted plane to 100 along the normal: new position = [227.   0.   0.]
Shifted plane to -100 along the normal: new position = [127.   0.   0.]
Labels
Sheet Seperations
flood_fill_layer
Shifted plane to 100 along the normal: new position = [227.   0.   0.]
Shifted plane to -100 along the normal: new position = [127.   0.   0.]
Labels
Sheet Seperations
flood_fill_layer
Shifted 2D slice to: 27
Shifted 2D slice to: 127
Labels
Sheet Seperations
flood_fill_layer
Shifted 2D slice to: 27
Shifted 2D slice to: 127
Labels
Sheet Seperations
flood_fill_layer
Shifted 2D slice to: 27
Shifted 2D slice to: 127
Labels
Sheet Seperations
flood_fill_layer
Shifted 2D slice to: 27
Shifted 2D slice to: 127
